# Similarity using doc2vec

In [2]:
# importing the necessary packages
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

os.chdir("C:\\Users\\SJNic\\Documents\\University\\Master\\Thesis\\Thesis Code")


lemmas = pd.read_pickle("Similarities/newLemmas.pkl")#.to_frame()
print(type(lemmas))

<class 'pandas.core.frame.DataFrame'>


In [3]:
lemmas

,lemmas
key,
0000a2fd-0427-4949-a3b4-e39f4635f421,"[cx, e, t, fric, cd, clliot, f, tc, apt, motie..."
0005b23b-f2be-4b55-9f83-dcf966cb3b29,[]
00068e84-19ef-4db9-8d2a-7e51ff69e18c,"[utrecht, amendement, vergadering, zijn, gemee..."
00096dc9-44fd-4ba2-aa89-6ece5f79a44b,"[motie, beargumenter, bezuinig, gemeenteraad, ..."
00097ca8-b918-4dd3-99bf-9728d1fd5ee4,"[bijlage, bizone, merwedekanaalzon, zuid]"
...,...
fff65d87-1368-42a0-85a0-028a36b66820,"[stadsontwikkeling, bijlage, bestemmingsplan, ..."
fff9faf4-7f86-43af-a065-d803b85e1268,"[verordening, upas, gemeente, utrecht, raad, g..."
fffe5196-feb7-4be1-8ef7-cc55a79e16ac,[]


In [6]:
tagged_data = [TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(lemmas.lemmas)]
model_d2v = Doc2Vec(vector_size=100,alpha=0.025, min_count=1)
model_d2v.build_vocab(tagged_data)

In [7]:
for epoch in tqdm(range(100)):
    model_d2v.train(tagged_data,
                total_examples=model_d2v.corpus_count,
                epochs=model_d2v.epochs)

100%|██████████| 100/100 [10:07:18<00:00, 364.39s/it]


In [8]:
document_embeddings=np.zeros((lemmas.shape[0],100))

In [9]:
document_embeddings.shape

(44613, 100)

In [10]:
for i in tqdm(range(len(document_embeddings))):
    document_embeddings[i]=model_d2v.docvecs[i]

  0%|          | 0/44613 [00:00<?, ?it/s]<ipython-input-10-a5baa0a14802>:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  document_embeddings[i]=model_d2v.docvecs[i]
100%|██████████| 44613/44613 [00:00<00:00, 222476.70it/s]


In [11]:
# saving the mebeddings
np.save("Similarities/document_embeddings.npy", document_embeddings)

In [3]:
# loading the embeddings
document_embeddings = np.load("Similarities/document_embeddings.npy")

# Pairwise similarities

In [12]:
pairwise_similarities=cosine_similarity(document_embeddings)

In [13]:
pairwise_similarities.shape

(44613, 44613)

In [20]:
#np.save("Similarities/pairwise_similarities", pairwise_similarities)

In [17]:
# function for calculating the similarity between two vectors
def most_similar(item_ID,similarity_matrix, max_items):
    print (f'Document: {lemmas.iloc[item_ID]["lemmas"][0:40]}')
    print ('Similar Documents:')
    
    most_related_items = []
    
    similar_ix=np.argsort(similarity_matrix[item_ID])[::-1]
    
    for ix in similar_ix[:max_items]:
        if ix==item_ID:
            continue
        docID = lemmas.index[ix]
        most_related_items.append(docID)
        print("______________________________________________________________________________________________")
        print(f"DOC ID: {docID}")
        print(f"DOC loc: {lemmas.index.get_loc(docID)}")
        print (f'Similarity score : {similarity_matrix[item_ID][ix]}')
        print (f'Text preview: {lemmas.iloc[ix]["lemmas"][0:40]}')
        
    return most_related_items

In [25]:
lemmas.index.get_loc("853cc5b5-f973-4a05-8045-b4e3319e3c91")

23028

In [17]:
for i in range(1000):
    #print(lemmas.iloc[i].lemmas)
    if len(lemmas.iloc[i].lemmas) > 1:
        print(i)
        print(lemmas.iloc[i].lemmas)
        break


90
['voorzitter', 'raad', 'fractie', 'cda', 'd66', 'gemeenteraad', 'oktober', 'debat', 'mogen', 'houden', 'feit', 'college', 'nouw', 'nieuw', 'invalsweg', 'a2', 'centrum', 'opnemen', 'actieplan', 'donderdag', 'september', 'college', 'actieplan', 'luchtkwaliteit', 'presenteren', 'plan', 'maatregel', 'luchtkwaliteit', 'verbeteren', 'groot', 'ruimtelijk', 'plan', 'studie', 'gemeente', 'komen', 'jaar', 'besluit', 'willen', 'nemen', 'leidsch', 'rijn', 'centrum', 'rijnenburg', 'actieplan', 'zijn', 'vrijblijvend', 'plan', 'gemeente', 'utrecht', 'actieplan', 'onderdeel', 'uitmaken', 'nationaal', 'samenwerkingsprogramma', 'luchtkwaliteit', 'nsl', 'maatregel', 'nsl', 'maatregel', 'actieplan', 'uitvoeringsplicht', 'project', 'actieplan', 'besluitvorming', 'plaats', 'kunnen', 'vinden', 'periode', 'half', 'oktober', 'gemeente', 'utrecht', 'ander', 'overheid', 'regio', 'vast', 'maatregel', 'project', 'indienen', 'nsl', 'project', 'opnemen', 'nsl', 'hoog', 'waarschijnlijk', 'doorgang', 'vinden', 'pro

In [26]:
most_similar(23028, pairwise_similarities, 10)

Document: ['utrecht', 'circulair', 'utrechtnl', 'circulair', 'utrecht', 'circulair', 'colofon', 'omslagfoto', 'cultuurvoorziening', 'berlijnplein', 'projectgroep', 'marieke', 'hoffmann', 'hele', 'yvanca', 'ike', 'demian', 'keetelaars', 'zeger', 'johan', 'boterenbrood', 'gerhard', 'frank', 'van', 'der', 'vaart', 'dank', 'intern', 'extern', 'betrekken', 'projectmanagement', 'wim', 'beelen', 'anouk', 'teuns', 'edwin', 'koster', 'grafisch', 'realisatie', 'ontwerpstudioruimte']
Similar Documents:
______________________________________________________________________________________________
DOC ID: 4c7275c1-50a6-4448-9462-8723ecfe85f2
DOC loc: 13282
Similarity score : 0.6892498725159937
Text preview: ['bijlage', 'inzet', 'stad', 'amsterdam', 'ambitie', 'koploper', 'transitie', 'circulair', 'economie', 'circulair', 'economie', 'zijn', 'rood', 'draad', 'overkoepelen', 'thema', 'agenda', 'duurzaam', 'maart', 'unaniem', 'zijn', 'vaststellen', 'gemeenteraad', 'amsterdam', 'zijn', 'stad', 'integra

['4c7275c1-50a6-4448-9462-8723ecfe85f2',
 '71e19129-1de6-43da-96dd-f71b13de2252',
 '5c7abeb6-41f1-4875-8e70-7e0f4fbc5622',
 '6303a678-c9ab-4e7f-a24a-442fee580ef5',
 '924dcc1c-19db-4a1d-ae6a-1f9d40404920',
 'f6666399-1d0c-4563-83a0-a2be500d42bf',
 '93d8384d-0925-4a9d-a82f-79bf862316dc',
 'e2e20a20-43e6-43fd-b876-3a1252f3a33b',
 'eb0aa161-015a-439e-8a7a-90ef83636c6d']